In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
import imodelsx.util
import sasc.viz
import pickle as pkl
from PIL import Image
import img2pdf
import json
from copy import deepcopy
from numpy.linalg import norm
from math import ceil
from sasc.config import CACHE_DIR, RESULTS_DIR, cache_ngrams_dir, regions_idxs_dir, FMRI_DIR
import sasc.modules.fmri_module
ngrams_list = joblib.load(join(cache_ngrams_dir, 'fmri_UTS02_ngrams.pkl')) # ngrams are same for both models

subject = 'S03'
# suffix_setting = '_fedorenko'
suffix_setting = '_spotlights'

if suffix_setting == '':
    # rois_dict = joblib.load(join(regions_idxs_dir, f'rois_{subject}.jbl'))
    # rois = joblib.load(join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/', 'communication_rois_UTS02.jbl'))
    rois = joblib.load(join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/',
                            f'communication_rois_v2_UT{subject}.jbl'))
    rois_dict_raw = {i: rois[i] for i in range(len(rois))}
    if subject == 'S02':
        raw_idxs = [
            [0, 7],
            [3, 4],
            [1, 5],
            [2, 6],
        ]
    elif subject == 'S03':
        raw_idxs = [
            [0, 7],
            [3, 4],
            [2, 5],
            [1, 6],
        ]
    rois_dict = {
        i: np.vstack([rois_dict_raw[j] for j in idxs]).sum(axis=0)
        for i, idxs in enumerate(raw_idxs)
    }
elif suffix_setting == '_fedorenko':
    if subject == 'S03':
        rois_fedorenko = joblib.load('lang_localizer_UTS03.jbl')
    rois_dict = {
        i: rois_fedorenko[i] for i in range(len(rois_fedorenko))
    }
    # rois_dict = rois_dict_raw
elif suffix_setting == '_spotlights':
    rois_spotlights = joblib.load('all_spotlights_UTS03.jbl')
    rois_dict = {i: rois_spotlights[i][-1] for i in range(len(rois_spotlights))}

### Visualize ROIs
Make sure these actually look right (esp. with contralateral regions)

In [ ]:
# # # save pcs
n_rois = min(8, len(rois_dict))
for i in tqdm(range(n_rois)):
    flatmap = rois_dict[i]
    # flatmap = np.zeros(weights_arr_full.shape[1])
    # flatmap[pfc] = pc_coefs_per_voxel[:, pc_num]
    sasc.viz._save_flatmap(
        flatmap, subject, fname_save=f'rois_custom_images/communication_{i}_{subject}.png')

# read all plots and save as subplots on the same page
C = 2
R = ceil(n_rois/C)
fig, axs = plt.subplots(R, C, figsize=(C * 4, R * 2))
axs = axs.ravel()
for i in range(n_rois):
    axs[i].imshow(Image.open(
        f'rois_custom_images/communication_{i}_{subject}.png'))
    axs[i].axis('off')
    axs[i].set_title(f'ROI {i}')
plt.savefig(f'communication_subplots_{subject}{suffix_setting}.png')

### Get predictions from embs
Run roi_custom_save_top_ngrams.py

### Load outputs

In [ ]:
def return_ranked_df(outputs_dict, rois_dict):
    outputs_dict_rank = {}
    for i, k in tqdm(enumerate(outputs_dict)):
        outputs_by_vox = np.vstack(outputs_dict[k])  # n_voxels x n_ngrams
        roi_mean = outputs_by_vox.mean(axis=0)
        roi_mean_other = np.mean([
            np.vstack([outputs_dict[j]])[0].mean(axis=0)
            for j in rois_dict.keys() if j != i
        ])

        # standard approach: use avg response by voxel
        # outputs_dict_rank[k] = roi_mean

        # use ranks by voxel rather than avg response by voxel
        # outputs_dict_rank[k] = np.argsort(outputs_by_vox, axis=1).mean(
        #     axis=0) * -1  # / outputs_by_vox.shape[1]

        # just output voxels no ranking
        vox_idxs = np.where(rois_dict[i])[0]
        for i in range(len(outputs_dict[k])):
            # voxel only
            # outputs_dict_rank[f'ROI{k}_vox{vox_idxs[i]}'] = outputs_dict[k][i]

            # subtract mean of roi from the individual voxel
            # outputs_dict_rank[f'ROI{k}_vox{vox_idxs[i]}'] = outputs_dict[k][i] - \
            # roi_mean

            # subtract mean of all other voxels in roi from the individual voxel
            outputs_dict_rank[f'ROI{k}_vox{vox_idxs[i]}'] = outputs_dict[k][i] - \
                roi_mean_other

    df = pd.DataFrame(outputs_dict_rank, index=ngrams_list)
    return df


rank_individual_voxels = False
if rank_individual_voxels:
    outputs_dict = joblib.load(
        join(cache_ngrams_dir, f'rois_communication_ngram_outputs_dict_voxels_{subject}{suffix_setting}_opt.pkl'))
    df_opt = return_ranked_df(outputs_dict, rois_dict)
    outputs_dict = joblib.load(
        join(cache_ngrams_dir, f'rois_communication_ngram_outputs_dict_voxels_{subject}{suffix_setting}_llama.pkl'))
    df_llama = return_ranked_df(outputs_dict, rois_dict)
    df = df_opt + df_llama
else:
    outputs_dict = joblib.load(
        join(cache_ngrams_dir, f'rois_communication_ngram_outputs_dict_{subject}{suffix_setting}_opt.pkl'))
    df_opt = pd.DataFrame(outputs_dict, index=ngrams_list)
    outputs_dict = joblib.load(
        join(cache_ngrams_dir, f'rois_communication_ngram_outputs_dict_{subject}{suffix_setting}_llama.pkl'))
    df_llama = pd.DataFrame(outputs_dict, index=ngrams_list)
    df = df_opt + df_llama

    # replace df values with ranks (here, higher is better, both before and after)
    df = df.rank(axis=0)

    ROI_NAMES = rois_dict.keys()
    for k in tqdm(ROI_NAMES):
        df_opt[str(k) + '_only'] = df_opt[k] - \
            df_opt[[c for c in ROI_NAMES if c != k]].mean(axis=1)
        df_llama[str(k) + '_only'] = df_llama[k] - \
            df_llama[[c for c in ROI_NAMES if c != k]].mean(axis=1)
        df[str(k) + '_only'] = df[k] - \
            df[[c for c in ROI_NAMES if c != k]].mean(axis=1)


stability_scores = {
    k: np.corrcoef(df_opt[k], df_llama[k])[0, 1]
    for k in df.columns
}

# get top ngrams (highest scores/ranks) for each ROI
top_ngrams_dict = {}
for k in tqdm(df.columns):
    top_ngrams_dict[k] = df[k].sort_values(
        ascending=False).index[:100].tolist()
top_ngrams_df = pd.DataFrame(top_ngrams_dict)
top_ngrams_df.to_csv(f'top_ngrams_by_roi_{subject}{suffix_setting}.csv')
with pd.option_context('display.max_rows', None):
    # rois = rois_dict.keys()
    # rois = [r for r in rois if not r == 'pSTS']  # never localized pSTS in S03
    display(top_ngrams_df.head(30))
top_ngrams_df.to_pickle(
    f'top_ngrams_custom_communication_{subject}{suffix_setting}.pkl')

In [ ]:
if suffix_setting == '_spotlights':
   # filter based on stability score
    STABILITY_THRESH = 0.7
    ks = [k for k in top_ngrams_df.columns if stability_scores[k] > STABILITY_THRESH]
    # ks = [k for k in ks if 'only' not in str(k)]
    ks = [k for k in ks if 'only' in str(k)]
    top_ngrams_df_filt = top_ngrams_df[ks]
    top_ngrams_df_filt.to_pickle(
        f'top_ngrams_custom_communication_{subject}{suffix_setting}_filtered.pkl')

In [ ]:
# # rois known (may use these as baselines)

# outputs_dict_known = joblib.load(
#     join(cache_ngrams_dir, f'rois_ngram_outputs_dict_{subject}_opt.pkl'))
# df_opt_known = pd.DataFrame(outputs_dict_known, index=ngrams_list)
# outputs_dict_known = joblib.load(
#     join(cache_ngrams_dir, f'rois_ngram_outputs_dict_{subject}_llama.pkl'))
# df_llama_known = pd.DataFrame(outputs_dict_known, index=ngrams_list)
# df_known = df_opt_known + df_llama_known
# top_ngrams_dict_known = {}
# for k in df_known.columns:
#     top_ngrams_dict_known[k] = df_known.sort_values(k).index[:100].tolist()
# top_ngrams_df_known = pd.DataFrame(top_ngrams_dict_known)

In [ ]:
# gpt4 = imodelsx.llm.get_llm('gpt-4-turbo-0125-spot')
# try:
#     explanations = json.load(open(f'explanations_by_roi_{subject}.json', 'r'))
# except:
#     explanations = {}
# for k in top_ngrams_df.columns:

#     s = '- ' + '\n- '.join(top_ngrams_df[k].iloc[:60])
#     prompt = f'''Here is a list of phrases:
#     {s}

#     What is a common theme among these phrases? Return only a concise phrase.'''
#     if not k in explanations:
#         explanations[k] = gpt4(prompt)
# # json.dump(explanations, open(
#     # f'explanations_by_roi_communication_{subject}.json', 'w'), indent=4)

In [ ]:
explanations = json.load(
    open(f'explanations_by_roi_communication_{subject}{suffix_setting}.json', 'r'))
# dataframe of explanations and stability scores
explanations_df = pd.DataFrame(explanations, index=['explanation']).T
explanations_df['explanation'] = explanations_df['explanation'].str.lower().str.replace(
    r'[^\w\s]', '').str.replace(r'\s+', ' ').str.strip()
stab = pd.Series(stability_scores)
# set index to al lbe strings
stab.index = stab.index.astype(str)
explanations_df['stability'] = stab

# add top ngrams
top_ngrams_list_list = top_ngrams_df.T.apply(lambda x: x.tolist(), axis=1)
top_ngrams_list_list.index = top_ngrams_list_list.index.astype(str)
explanations_df['top_ngrams'] = top_ngrams_list_list
explanations_df.to_csv(
    f'communication_explanations_{subject}{suffix_setting}.csv')

with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(explanations_df)

In [ ]:
# lowercase, remove punctuation, trim
explanations_df['explanation'] = explanations_df['explanation'].str.lower().str.replace(
    r'[^\w\s]', '').str.replace(r'\s+', ' ').str.strip()
explanations_df['explanation'].value_counts().head(50)
# split index on _
explanations_df['roi'] = explanations_df.index.str.split('_').str[0]
# groupby roi and show most common explanations
# display all
with pd.option_context('display.max_rows', None):
    display(explanations_df.groupby('roi')[
            'explanation'].value_counts().groupby('roi').head(5))
# explanations_df.groupby('roi')['explanation'].value_counts().groupby('roi').head(50)
# explanations_df.to_csv(f'communication_explanations_{subject}.csv')

### S03 Export selected rois to pkl

In [ ]:
rois = ['RSC', 'OPA', 'PPA', 'IPS', 'sPMv', 'EBA', 'OFA'] + \
    ['RSC_only', 'OPA_only', 'PPA_only']  # all but 'pSTS'
# pprint({k: explanations[k] for k in rois})
explanations_clean = {
    'EBA': 'Body parts',
    'IPS': 'Descriptive elements of scenes or objects',
    # OFA differs from UTS02 (which was "'Personal growth and reflection',")
    'OFA': 'Conversational transitions',
    'OPA': 'Direction and location descriptions',
    # OPA_only differs from UTS02 (which was 'Spatial positioning and directions')
    'OPA_only': 'Self-reflection and growth',
    'PPA': 'Scenes and settings',
    'PPA_only': 'Unappetizing foods',
    'RSC': 'Travel and location names',
    'RSC_only': 'Location names',
    # sPMv differs from UTS02 (which was 'Time and Numbers')
    'sPMv': 'Dialogue and responses',
}
explanation_avoid_suffixes = {
    'EBA': ' Avoid mentioning any locations.',
    'IPS': ' Avoid mentioning any locations.',
    'OFA': ' Avoid mentioning any locations.',
    'OPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'OPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'RSC': '',
    'RSC_only': '',
    'sPMv': ' Avoid mentioning any locations.'
}
for roi in rois:
    print(f'"{roi}":', str(
        top_ngrams_df[roi.replace('1', '').replace('2', '')].iloc[:50].values.tolist()) + ', ')
top_ngrams_clean = {
    "RSC": ['was led upstairs', 'onto the subway', 'to the hallway', 'drove to washington', 'back through london', 'and darted downstairs', 'past the offices', 'long hallway toward', 'down the sidewalk', 'back in manhattan', 'reached the interstate', 'just blocks away', 'drove from vermont', 'was standing outside', 'to a courtyard', 'in the alley', 'up the coast', 'from my dorm', 'in the courtyard', 'in central park', 'i walk outside', 'here in manhattan', 'darted downstairs', 'facing the beach', 'walk through downtown', 'wander the hallways', 'i ran downstairs', 'down the hall', "'m standing outside", 'off into vancouver', 'through the streets', 'sitting in indianapolis', 'on sixth avenue', 'i go upstairs', 'across the street', 'arrived in indianapolis', 'we were downtown'],
    "OPA": ['railing looking out', 'across a plateau', 'up the coast', 'against the railing', 'in the courtyard', 'up the hill', 'above the gulf', 'outside the windows', 'long hallway toward', 'over the gulf', 'past the offices', 'through the windows', 'beside the river', 'past the waterfall', 'across the bridge', 'this long hallway', 'to a courtyard', 'and the courtyard', 'and behind me', 'down this embankment', 'towards the river', 'the hill up', 'courtyard was surrounded', 'in an alcove', 'onto the railing', 'along the coast', 'up the stairs', 'across the quadrangle', 'facing the beach', 'to the north', 'down the corridor', 'through the gates', 'over the embankment', 'onto the bridge', 'down that corridor', 'down the sidewalk', 'i looked across', 'path that jutted', 'through this door', 'the lagoon behind', 'down the embankment', 'on the railing', 'on the embankment', 'through the doors', 'on the windowsill', 'corridor out onto', 'the buildings beside', 'to the hallway', 'by that window', 'past the city', 'door behind me', 'to the south', 'off the coast', 'cross the bering', 'around the reef', 'behind me i', 'driveway and behind', 'against the windows', 'across the street', 'to the shoreline', 'lagoon behind the', 'on the sidewalk', 'hall past the', 'off the east', 'of the ravine', 'surrounded the city', 'in the window', 'southern shore of', 'in the distance', 'onto the sidewalk', 'i look across', 'behind us i', 'behind us there', 'on the cliff', 'over the river', 'toward the ocean', 'on that terrace', 'row of stalls', 'sidewalk in front', 'down the long', 'on the walls', 'door to the', 'by the window', 'outside my door', 'outside the door', 'across from me', 'on the eastern', 'the hall past', 'down the lagoon', 'in the forest', 'that window in', 'around me the', 'to the barrier', 'the gulf where', 'road in front', 'in the hallway', 'across the parking', 'in the colonnade', 'to the western', 'surrounded by rooms'],
    "PPA": ['in an alcove', 'on the stoop', 'past the offices', 'against the railing', 'on the windowsill', 'in the alley', 'to a courtyard', 'the copier room', 'in the courtyard', 'this long hallway', 'to the hallway', 'on a dock', 'in the hallway', 'long hallway toward', 'outside the windows', 'on that terrace', 'inside the hut', 'railing looking out', 'through the windows', 'down this embankment', 'on the subway', 'onto the subway', 'there were shelves', 'in my cubicle', 'a strip mall', 'on the sidewalk', 'in the colonnade', 'on the railing', 'into the basement', 'across the parking', 'a restaurant stoop', 'onto the railing', 'exit the subway', 'by the window', 'in that attic', 'was led upstairs', 'in the basement', 'the food court', 'and the courtyard', 'in the cafeteria', 'hall past the', 'into the parking', 'in the windowless', 'back room where', 'on my bed', 'down the sidewalk', 'contain strip malls', 'onto the sidewalk', 'the hall closet', 'at those cliffs'],

    "RSC_only": ['moved to chicago', 'drove from vermont', 'came to florida', 'here in manhattan', 'living in chicago',  'move to texas', 'leaving for france', 'back in manhattan', 'to boston to', 'went to boston', 'moved to vermont', 'geese in ohio', 'college in boston', 'in ohio', 'moved to brooklyn', 'normal suburban pittsburgh', 'moved to london', 'back in israel', 'to london to', 'come from israel', 'went to manchester', 'to columbus ohio', 'here in boston', 'i left vermont', 'from pittsburgh pennsylvania', 'in lower manhattan', 'hometown in texas', 'touring through europe', 'in warmer mexico', 'union in manhattan', 'suburban pittsburgh', 'moved to washington', 'was in boston', 'slacking in madison', 'chick from silverlake', 'heading to iraq', 'in chicago', 'in louisville kentucky', 'lived in hiroshima', 'in florida'],
    "OPA_only": ['eventually i forgave', 'push past it', 'eventually forgave', 'she eventually forgave', 'i forgave', 'to see ourselves', 'of myself which', 'means extending empathy', 'forgive and', 'i stopped myself', 'forgive and love', 'of the hurt', 'i rise above', 'i pushed myself', 'is and who', "'m hurt but", 'see ourselves and', 'the hurt', 'i persisted and', 'to forgive afterwards', 'was real to', 'comparing myself', 'looked in myself', 'selves which', 'around it and', 'self which translated', 'inside me that', 'of me which', 'overcome my ambivalence', 'to push myself', 'was also influenced', 'the side which', 'side which', 'independence and freedom'],
    "PPA_only": ['a garbage bag', 'that garbage bag', "'re throwing napkins", 'box of discarded', 'in sugar jars', 'their chew toys', 'those plastic containers', 'our dishwasher', 'skivvies toothbrush floss', 'of cheap beer', 'throwing napkins', 'a trash can', 'milk bottle tops', 'want a mcflurry', 'vomit smelling couch', 'grown napkins', 'my cheese sandwich', 'overpriced coffee shops', 'some lighter fluid', 'salad and stale', 'vomited a washpan', 'these brown paper'],

    "IPS": ['and behind me', 'against the railing', 'onto the railing', 'path that jutted', 'situated herself behind', 'above the gulf', 'door behind me', 'southern shore of', 'i looked across', 'along the edge', 'closed behind me', 'behind me and', 'across a plateau', 'on the railing', 'up behind me', 'leaning against the', 'towering above me', 'jutted into the', 'onto the bridge', 'and cut across', 'behind him and', 'and came around', 'front of us', 'up onto the', 'over the gulf', 'stood behind me', 'across the bridge', 'beside the river'],
    "sPMv": ['repeated her affirmation', 'said excuse me', 'asked i laughed', 'and goes hey', 'response was nah', 'hurry she exclaimed', 'said no i', 'just nodded yes', 'retorted rather loudly', 'was like hey', 'called her and', 'and said yes', 'and screams fuck', 'said uh hey', 'says sure and', 'says uh actually', 'was like hi', 'i said fine', 'said without hesitation', 'said well yes', 'says excuse me', 'i asked immediately', 'she yelled i', 'said mom mom', 'said did i', 'i said wow', 'i said shyly', 'asked her and', 'said okay okay', 'i sheepishly raise', 'which i responded', 'turned and said', 'then wrote yes', 'said yes i', 'whisper she said', 'was like mhm'],
    "EBA": ['arms around her', 'wraps his arms', 'hands gripped the', 'into my palm', 'hands into my', 'elbows on knees', 'grab his arms', 'his hands folded', 'into her arms', 'grabbed her hand', 'arms flailing', 'grabbed her legs', 'arm around my', 'grabbed their hands', 'lifted her up', 'put my arms', 'leaned his head', 'put his arms', 'shakes my hand', 'flying arms flailing', 'i leaned down', 'arms tighten around', 'her hands gripped', 'hand on his', 'my feet kicking', 'pinning my arms', 'held her hand', 'in a headlock', 'pressed my face', 'holds her hand', 'arms flailing holding', 'rubbing his head'],
    "OFA": ['and we talked', 'i even met', 'and so finally', 'one night my', 'one evening after', 'anyway the point', 'one summer my', 'weeks passed and', 'finally one day', 'then we talked', 'one night i', 'we chatted', 'and i talked', 'we talked and', 'talked and', 'so i texted', 'to my surprise', 'one afternoon when', 'i persisted and', 'and i finally', 'was watching television', 'i remember once', 'so one night', 'but anyway', 'and i met', 'when i finally', 'so i finally', 'on and on', 'son had finally', 'and i especially', 'so one day', 'and eventually i', 'we brace ourselves', 'so anyway', 'i was perusing', 'and it finally', 'later that day', 'so we finally', 'but anyhow', 'and as we', 'the day came', 'home one afternoon', 'then i finally', 'what fascinated me', 'and i vaguely', 'so i talked', 'once while i', 'was hanging out', 'i was reliving', 'but the most'],
}

rows = {
    'roi': rois,
    'expl': [explanations_clean[k] for k in rois],
    'top_ngrams_module_correct': [top_ngrams_clean[k] for k in rois],
    'stability_score': [stability_scores[k.split('_')[0]] for k in rois],
    'subject': [f'UT{subject}'] * len(rois),
    'voxel_nums': [rois_dict[k.split('_')[0]] for k in rois],
    'prompt_suffix': [explanation_avoid_suffixes[k] for k in rois],
}
rows = pd.DataFrame(rows)
rows.to_pickle(f'rows_roi_ut{subject.lower()}_may31.pkl')

### S02 Export selected rois to pkl

In [ ]:
rois = ['RSC', 'OPA', 'PPA', 'IPS', 'pSTS', 'sPMv',
        'EBA', 'OFA'] + ['RSC_only', 'OPA_only', 'PPA_only2']  # 'PPA_only1',
# pprint({k: explanations[k] for k in rois})
explanations_clean = {
    'EBA': 'Body parts',
    'IPS': 'Descriptive elements of scenes or objects',
    'OFA': 'Personal growth and reflection',
    'OPA': 'Direction and location descriptions',
    'OPA_only': 'Spatial positioning and directions',
    'PPA': 'Scenes and settings',
    'PPA_only': 'Unappetizing foods',
    'RSC': 'Travel and location names',
    'RSC_only': 'Location names',
    'pSTS': 'Verbal interactions',
    'sPMv': 'Time and numbers'}
explanation_avoid_suffixes = {
    'EBA': ' Avoid mentioning any locations.',
    'IPS': ' Avoid mentioning any locations.',
    'OFA': ' Avoid mentioning any locations.',
    'OPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'OPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'RSC': '',
    'RSC_only': '',
    'pSTS': ' Avoid mentioning any locations.',
    'sPMv': ' Avoid mentioning any locations.'
}
for roi in rois:
    print(f'"{roi}":', str(
        top_ngrams_df[roi.replace('1', '').replace('2', '')].iloc[:50].values.tolist()) + ', ')
    # {
    # roi:  for roi in rois
# })
top_ngrams_clean = {
    "RSC": ['drove from vermont', 'to washington', 'in manhattan', 'here in boston', 'off into vancouver', 'moved to chicago', 'was in mexico', 'arrived in indianapolis', 'came to florida', 'i left vermont'],
    "OPA": ['onto the railing', 'towards the river', 'onto the sidewalk', 'towards the doors', 'outside the windows', 'long hallway toward', 'to the horizon', 'towards the street', 'over the gulf', 'to my left', 'path that jutted', 'on the ceiling', 'on the windowsill', 'down this embankment', 'up those stairs', 'above the gulf', 'facing the beach'],
    "PPA": ['mile of cornfields', 'the windowsill', 'the rolling hills', 'beautiful moonlit mountains', 'giant stone cliffs', 'a strip mall', 'nondescript office buildings', 'manicured lawns', 'lakes', 'the dark driveway', 'and shimmering skyscrapers', 'a private beach', 'the leafy garden', 'our modest backyard', 'my dorm'],

    "RSC_only": ['florida', 'israel', 'london', 'marrakesh', 'indianapolis', 'paris', 'pennsylvania', 'tokyo', 'tenessee', 'boston', 'vermont', 'chicago', 'indianapolis'],
    "OPA_only": ['towards the ceiling', 'onto the railing', 'feet hanging over', 'towards the doors', 'seats behind', 'towards the door', 'lights peeking over', 'to my left', 'situated herself behind', 'you sit backward', 'to the horizon', 'maybe twelve feet', 'at the ceiling', 'towards the street', 'of seats behind', 'twenty feet above', 'his back turned', 'see the horizon', 'seats behind the', 'to my right', 'and high rafters', 'about twenty feet', 'door behind me', 'the door behind', 'toward the back', 'over his shoulder', 'feet above the', 'hands went underneath', 'towards the ground', 'his feet hanging', 'feet touch the', 'behind her and', 'stand in front', 'down one side', 'on opposite sides', 'over the ceiling', 'on either side'],
    # "PPA_only": ['kind of corny', 'his painting sucked', 'snake oil', 'liar fake', 'fake name', 'bad puns', 'as an insult', 'called baloney'],
    "PPA_only2": ['like burnt steak', 'like pudding', 'tasted pretty bad', 'stale baked goods', 'the crusts', 'baloney', 'yeast extract', 'a sandwich rejected',],

    "IPS": ['there were slats', 'four connected squares', 'in long rows', 'on the sides', 'a long narrow', 'that forms horizontal', 'long rows of', 'sixty foot wide', 'between buttered slices', 'mile thick ice', 'all four corners', 'along the top'],
    "pSTS": ['said excuse me', 'says excuse me', 'room went silent', 'someone shouted', 'i provoked gasps', 'somebody then yelled', 'she started laughing', 'excuse me', 'asked i laughed', 'exhalation someone shouted', 'retorted rather loudly', 'turned and said', 'hurry she exclaimed', 'i started yelling', 'say excuse me', 'i started laughing', 'interrupted the conversation', 'breath he yelled', 'moment she gasped', 'said guess what'],
    "sPMv": ['one', 'forty', 'april nineteen forty', 'was sixteen seventeen', 'five only twenty', 'three down', 'march twentieth nineteen', 'more time passed', 'fifteen meters fifty', "turning ninety", 'june of nineteen'],
    "EBA": ['wraps his arms', 'lifted her dress', 'arms flailing', 'hands gripped the', 'grabbed her legs', 'his hands folded', 'my feet kicking', 'navigated pushy elbows', 'elbows on knees', 'over his shoulder'],
    "OFA": ['of my childhood', 'newfound self esteem', 'so my shrink', 'hurtful first dates', 'recall many instances', 'it felt magical', 'answered many questions', 'my school days', 'no satisfying fantasies', 'my mom often', 'from our childhood', 'growing up we', 'good friends often', 'shaped their mind', 'everything my parents'],
}

rows = {
    'roi': rois,
    'expl': [explanations_clean[k] for k in rois],
    'top_ngrams_module_correct': [top_ngrams_clean[k] for k in rois],
    'stability_score': [stability_scores[k.split('_')[0]] for k in rois],
    'subject': [f'UT{subject}'] * len(rois),
    'voxel_nums': [rois_dict[k.split('_')[0]] for k in rois],
    'prompt_suffix': [explanation_avoid_suffixes[k] for k in rois],
}
rows = pd.DataFrame(rows)
rows.to_pickle(f'rows_roi_ut{subject.lower()}_may31.pkl')